In [176]:
import pandas as pd
import numpy as np
df_log_in=pd.read_csv('log_in_table.csv')
df_order=pd.read_csv('order_table_50k_v2.csv')
df_transaction_data=pd.read_csv('transaction_data.csv')

# Application Fraud Prevention

In [79]:
df_log_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50861 entries, 0 to 50860
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      50861 non-null  int64 
 1   log_in_date  50861 non-null  object
 2   device_id    50861 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [182]:
#是否存在一个device_id关联多个用户的情况
df_device_cnt=df_log_in.groupby("device_id").agg(user_cnt=("user_id", "size"),
                         ).reset_index()
df_device_fraud=df_device_cnt[df_device_cnt['user_cnt']>2]
df_device_fraud

,device_id,user_cnt
21013,6af890a803027331e93cc4d4d78653999,16


In [188]:
df_fraud_user_id

,user_id,log_in_date,device_id
118,90032,2010/1/22,6af890a803027331e93cc4d4d78653999
126,90039,2009/11/25,6af890a803027331e93cc4d4d78653999
132,90044,2009/6/19,6af890a803027331e93cc4d4d78653999
140,90051,2009/7/19,6af890a803027331e93cc4d4d78653999
145,90055,2009/10/19,6af890a803027331e93cc4d4d78653999
147,90056,2009/8/27,6af890a803027331e93cc4d4d78653999
149,90057,2009/1/9,6af890a803027331e93cc4d4d78653999
152,90059,2009/12/10,6af890a803027331e93cc4d4d78653999
17043,106153,2009/12/10,6af890a803027331e93cc4d4d78653999
17501,106610,2009/2/19,6af890a803027331e93cc4d4d78653999


In [191]:
#关联欺诈用户到订单表
df_fraud_user_id=df_log_in[df_log_in['device_id']=='6af890a803027331e93cc4d4d78653999']
df_order_2=pd.merge(df_order,df_fraud_user_id,how='left',on='user_id')
df_order_2['fraud_s1']=np.where(df_order_2.device_id.isnull()==False,1,0)

In [196]:
#套入分析框架
df=df_order_2
df_pivot=df[df['control_group']==0].groupby("fraud_s1").agg(order_cnt=("order_id", "size"),
                         approval_cnt=("order_status", "sum"),
                         fpd30=("fpd30", "sum"),
                         mob3_m2=("mob3_m2", "sum"),
                         principal=("principal", "sum")).reset_index()
df_pivot['fpd30_rate']=df_pivot['fpd30']/df_pivot['approval_cnt']
df_pivot['mob3_m2_rate']=df_pivot['mob3_m2']/df_pivot['approval_cnt']
df_pivot

,fraud_s1,order_cnt,approval_cnt,fpd30,mob3_m2,principal,fpd30_rate,mob3_m2_rate
0,0,44986,27659,105,464,93794300,0.003796,0.016776
1,1,14,6,6,6,17400,1.000000,1.000000


# Transaction Fraud Prevention
1.计算还有多少用户有潜在被盗刷的风险
2.计算这些用户潜在被盗刷的金额是多少

In [230]:
df_transaction_data[' amount ']

0        45.59
1        39.69
2        49.76
3        49.07
4        57.20
         ...  
19995    56.38
19996    34.93
19997    34.18
19998    58.91
19999    46.59
Name:  amount , Length: 20000, dtype: float64

In [214]:
df_transaction_data.columns

Index(['transaction_id', 'user_id', ' amount ', 'pos_id', 'transaction_date'], dtype='object')

In [90]:
df_transaction_data.describe()

,transaction_id,user_id,amount
count,20000.000000,20000.000000,20000.000000
mean,90330.000000,90414.500000,49.849176
std,5779.623715,288.682207,10.031318
min,79881.000000,89915.000000,13.170000
25%,85350.500000,90164.750000,43.037500
50%,90334.500000,90414.500000,49.900000
75%,95307.250000,90664.250000,56.600000
max,100803.000000,90914.000000,85.930000


In [233]:
#查看投诉用户交易信息
df_transaction_data_fraud=df_transaction_data[(df_transaction_data['user_id']==90559)
|(df_transaction_data['user_id']==90059)
|(df_transaction_data['user_id']==90222)
|(df_transaction_data['user_id']==90745)]
df_transaction_data_fraud

,transaction_id,user_id,amount,pos_id,transaction_date
115,80106,90059,50.88,e776843ee6f5ce7768436843ee6f5c,2009-01-03
131,80676,90059,51.76,d1c5c7351fb08d1c5c735c7351fb08,2009-01-14
152,81426,90059,59.35,f43c54892fe7ff43c548c54892fe7f,2009-01-29
186,82559,90059,45.48,42fb5585b82da42fb558b5585b82da,2009-02-20
264,82765,90059,59.20,b5585b82da42fb5585ba85b82da42f,2009-02-24
...,...,...,...,...,...
19279,96319,90745,45.64,9c6cf15c50de89c6cf19cf15c50de8,2009-11-08
19466,98482,90745,58.00,d4f6738116b40d4f67326738116b40,2009-12-21
19530,99184,90745,63.96,f2ed520897f38f2ed520d520897678,2010-01-04
19618,100127,90745,45.56,c458495175b64c4584958495175b64,2010-01-25


In [235]:
#寻找交易信息共性
df_pos_id=df_transaction_data_fraud.groupby("pos_id").agg(user_id=("user_id", "size"))
df_tmp=df_pos_id[df_pos_id['user_id']>1]
df_tmp

,user_id
pos_id,
f2ed520897f38f2ed520d520897678,4


In [237]:
#查看明细，还原案件
df_tmp=df_transaction_data_fraud[df_transaction_data_fraud['user_id']==90559]
df_tmp

,transaction_id,user_id,amount,pos_id,transaction_date
10407,80590,90559,44.56,789c319db5864789c31fc319db5864,2009-01-02
10964,81369,90559,41.40,27c68a2a6d50a27c68af68a2a6d50a,2009-01-17
11037,83590,90559,38.37,c297629920fb5c2976297629920fb5,2009-03-03
11155,84365,90559,60.43,50a27c68a2a6d50a27c627c68a2a6d,2009-03-19
11158,85040,90559,33.72,864789c319db5864789c789c319db5,2009-04-01
11599,87479,90559,61.69,f2ed520897f38f2ed520d520897678,2009-05-19
11684,87605,90559,38.24,7629920fb5c29762992a9920fb5c29,2009-05-21
11849,87677,90559,36.13,0ef8c8a528b980ef8c8a8c8a528b98,2009-05-23
12046,88079,90559,37.63,84511a4d8037684511a911a4d80376,2009-05-31
12210,88447,90559,43.66,b4974aca03d6fb4974ac74aca03d6f,2009-06-08


In [239]:
#预估损失#找到所有涉及这个posid的交易
df_transaction_data_fraud_all=df_transaction_data[(df_transaction_data['user_id']!=90559)
&(df_transaction_data['user_id']!=90059)
&(df_transaction_data['user_id']!=90222)
&(df_transaction_data['user_id']!=90745)
&(df_transaction_data['pos_id']=='f2ed520897f38f2ed520d520897678')]
display(df_transaction_data_fraud_all)
##user_id 去重计数
print(len(df_transaction_data_fraud_all['user_id'].unique()))

,transaction_id,user_id,amount,pos_id,transaction_date
202,80059,89941,54.98,f2ed520897f38f2ed520d520897678,2009-01-04
586,81155,90653,60.52,f2ed520897f38f2ed520d520897678,2009-01-11
762,80633,89955,29.62,f2ed520897f38f2ed520d520897678,2009-01-15
933,81054,90205,44.77,f2ed520897f38f2ed520d520897678,2009-01-18
1016,81345,90413,62.41,f2ed520897f38f2ed520d520897678,2009-01-20
1666,81980,90398,36.73,f2ed520897f38f2ed520d520897678,2009-02-02


6


In [241]:
df_f_all

,transaction_id_x,user_id,amount _x,pos_id_x,transaction_date_x,transaction_id_y,amount _y,pos_id_y,transaction_date_y
0,80374,90458,45.59,fd15d3183f0b9fd15d315d3183f0b9,2009-01-01,NaN,NaN,NaN,NaN
1,80673,90756,39.69,727f0374a06a4727f037f0374a06a4,2009-01-01,NaN,NaN,NaN,NaN
2,80217,90299,49.76,00bc8f466315f00bc8f4c8f466315f,2009-01-01,NaN,NaN,NaN,NaN
3,80251,90332,49.07,c8b5de39a104fc8b5de35de39a104f,2009-01-01,NaN,NaN,NaN,NaN
4,80377,90457,57.20,eaa4682ec62ddeaa46824682ec62dd,2009-01-01,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
19995,100619,90708,56.38,2ea26b3a2263a2ea26b126b3a2263a,2010-02-04,NaN,NaN,NaN,NaN
19996,100642,90730,34.93,541feac7a1b28541feabfeac7a1b28,2010-02-04,NaN,NaN,NaN,NaN
19997,100771,90858,34.18,75bb9ca0b5a0f75bb9c3b9ca0b5a0f,2010-02-04,NaN,NaN,NaN,NaN
19998,100512,90598,58.91,52b9ccf340b4c52b9cc59ccf340b4c,2010-02-04,NaN,NaN,NaN,NaN


In [243]:
#预估损失#统计这些用户在交易时间之后所有的金额
df_f_all=pd.merge(df_transaction_data,df_transaction_data_fraud_all
                                             ,how='left',on='user_id')
df_f_all.head()
df_f_all_loss=df_f_all[(df_f_all['transaction_date_y'].isnull()==False)\
             &(df_f_all['transaction_date_y']<df_f_all['transaction_date_x'])]
df_f_all_loss.describe()


,transaction_id_x,user_id,amount _x,transaction_id_y,amount _y
count,108.000000,108.000000,108.000000,108.000000,108.000000
mean,90800.601852,90264.981481,50.716389,81038.601852,48.317500
std,5257.187121,260.365989,9.190361,597.016147,12.276577
min,81464.000000,89941.000000,29.910000,80059.000000,29.620000
25%,86209.000000,89955.000000,44.195000,80633.000000,36.730000
50%,90706.500000,90398.000000,50.480000,81155.000000,54.980000
75%,95608.250000,90413.000000,57.250000,81345.000000,60.520000
max,100368.000000,90653.000000,69.190000,81980.000000,62.410000


In [249]:
df_f_all_loss[' amount _x'].sum()

5477.37

In [247]:
df_f_all_loss

,transaction_id_x,user_id,amount _x,pos_id_x,transaction_date_x,transaction_id_y,amount _y,pos_id_y,transaction_date_y
1135,81464,90413,39.62,950f00b0748c9950f006f00b0748c9,2009-01-22,81345.0,62.41,f2ed520897f38f2ed520d520897678,2009-01-20
1552,81673,90205,44.18,69e7e9b06a87469e7e9c7e9b06a874,2009-01-30,81054.0,44.77,f2ed520897f38f2ed520d520897678,2009-01-18
1786,81643,89941,68.07,811f549fc3472811f549f549fc3472,2009-02-04,80059.0,54.98,f2ed520897f38f2ed520d520897678,2009-01-04
1963,81820,89941,50.49,593c79c0cc6bc593c79dc79c0cc6bc,2009-02-07,80059.0,54.98,f2ed520897f38f2ed520d520897678,2009-01-04
2215,82544,90413,48.09,a44d59cb95323a44d59cd59cb95323,2009-02-13,81345.0,62.41,f2ed520897f38f2ed520d520897678,2009-01-20
...,...,...,...,...,...,...,...,...,...
18255,98126,89955,46.91,a66fa454dbabea66fa45fa454dbabe,2009-12-30,80633.0,29.62,f2ed520897f38f2ed520d520897678,2009-01-15
18381,98252,89955,40.16,c1f43ea1f2efcc1f43eb43ea1f2efc,2010-01-01,80633.0,29.62,f2ed520897f38f2ed520d520897678,2009-01-15
19123,99692,90653,52.79,f88d51c3b58e8f88d51cd51c3b58e8,2010-01-17,81155.0,60.52,f2ed520897f38f2ed520d520897678,2009-01-11
19373,99702,90413,56.84,6b95a5d6764c46b95a535a5d6764c4,2010-01-22,81345.0,62.41,f2ed520897f38f2ed520d520897678,2009-01-20
